## 사용 사례: 하이브리드 Deployment 아키텍처

### 시나리오: 인터넷 액세스를 통한 안전한 데이터 처리

이 deployment 패턴에서는 보안과 기능의 균형을 맞추기 위해 AgentCore 컴포넌트를 분리합니다:

**AgentCore Browser (Public 모드)**
- 인터넷 액세스가 가능한 퍼블릭 서브넷에 배포
- 공개적으로 액세스 가능한 웹사이트 및 서비스의 웹 브라우징 활성화
- 외부 통합 및 실시간 데이터 검색 관리
- 실시간 작업을 위한 직접 인터넷 연결의 이점 활용

**AgentCore Runtime (VPC 모드)**
- 보안 VPC 내의 프라이빗 서브넷에 배포
- 민감한 데이터 및 비즈니스 로직 처리
- 엄격한 네트워크 격리 유지
- 보안 내부 채널을 통해 browser 컴포넌트와 통신

### 이점

- **보안**: 민감한 처리는 프라이빗 네트워크에서 격리된 상태로 유지
- **성능**: Browser 작업이 NAT 오버헤드 없이 직접 인터넷 액세스 가능
- **규정 준수**: 데이터 격리에 대한 규제 요구 사항 충족
- **확장성**: 각 컴포넌트가 워크로드 요구 사항에 따라 독립적으로 확장 가능

### 아키텍처 흐름

![image.png](architecture-browser.png)

# CloudFormation Stack 실행 지침

## 사전 요구 사항
- 적절한 권한으로 구성된 AWS CLI
- CloudFormation 템플릿 파일 준비

## 실행 단계

### 1. 아래 단계를 실행하여 CFN 시작 (약 10분)

In [ ]:
import boto3
import yaml
import os

# 설정 변수
region = 'us-east-1'  # 원하는 리전으로 변경
template_path = 'cfn-browser.yaml'
stack_name = 'browser-stack'

# CloudFormation 클라이언트 초기화
cf_client = boto3.client('cloudformation', region_name=region)

# CloudFormation 템플릿 파일 읽기
with open(template_path, 'r') as template_file:
    template_body = template_file.read()

try:
    # CloudFormation 스택 생성
    response = cf_client.create_stack(
        StackName=stack_name,
        TemplateBody=template_body,
        Capabilities=['CAPABILITY_IAM', 'CAPABILITY_NAMED_IAM']  # IAM 리소스 생성 권한 필요
    )
    
    print(f"Stack creation initiated in region: {region}")
    print(f"Stack ID: {response['StackId']}")
    
    # 스택 생성 완료 대기 (약 10분 소요)
    waiter = cf_client.get_waiter('stack_create_complete')
    print("Waiting for stack creation to complete...")
    waiter.wait(StackName=stack_name)
    
    print(f"Stack '{stack_name}' created successfully in {region}!")
    
except Exception as e:
    print(f"Error creating stack: {str(e)}")

### 2. 테스트 지침

In [ ]:
import boto3
import subprocess
import json
from IPython.display import display, Markdown

# CloudFormation 스택의 Output 값을 가져오는 함수
def get_cfn_output(stack_name, output_key, region='us-east-1'):
    """CloudFormation 스택의 Output 값 조회"""
    cfn = boto3.client('cloudformation', region_name=region)
    try:
        response = cfn.describe_stacks(StackName=stack_name)
        outputs = response['Stacks'][0]['Outputs']
        for output in outputs:
            if output['OutputKey'] == output_key:
                return output['OutputValue']
    except Exception as e:
        print(f"Error fetching CFN output: {e}")
    return None

# 이전 셀에서 생성한 스택으로부터 필요한 값들 가져오기
agent_runtime_arn = get_cfn_output(stack_name, 'AgentRuntimeArn')
agent_runtime_id = get_cfn_output(stack_name, 'AgentRuntimeId')
development_instance = get_cfn_output(stack_name, 'DevelopmentInstanceId')


# 테스트 지침 생성
instructions = f"""# Testing Instructions for Bedrock Agent Runtime

## Prerequisites
- CloudFormation stack has been completed successfully

## Step-by-Step Testing Process

### 1. Connect to EC2 Instance
Connect to EC2 instance `{development_instance}` via Browser Connector or SSH

### 2. Setup Environment
Run the following commands on the EC2 instance:

```bash
sudo yum update -y

# Install development tools
sudo dnf install git -y && \\
curl -LsSf https://astral.sh/uv/install.sh | sh && \\
echo 'export PATH="$HOME/.cargo/bin:$PATH"' >> ~/.bashrc && \\
source ~/.bashrc && \\
echo 'Install Python Venv'

uv init vpc-browser --python 3.13 && cd vpc-browser
uv venv --python 3.13
source .venv/bin/activate
uv pip install boto3

cat > call-agent.py << 'EOF'
import boto3
import json

# Bedrock AgentCore 클라이언트 생성
client = boto3.client('bedrock-agentcore', region_name='us-east-1')

# Agent에게 전달할 프롬프트
payload = json.dumps({{
    "prompt": "What is the Weather in Richmond VA Today?"
}})

# Agent Runtime 호출
response = client.invoke_agent_runtime(
    agentRuntimeArn="{agent_runtime_arn}",
    runtimeSessionId='dfmeoagmreaklgmrkleafremoigrmtesogmtrskhmtkrlshmt',  # 33자 이상 필수
    payload=payload,
    qualifier="DEFAULT"
)

# 응답 파싱 및 출력
response_body = response['response'].read()
response_data = json.loads(response_body)
print("Agent Response:", response_data)
EOF


```

### 3 Run the agent test
```
python call-agent.py
```

### 4. Monitor logs in CloudWatch:
- Navigate to CloudWatch Logs in the AWS Console
- Look for log group: /aws/bedrock-agentcore/runtimes/{agent_runtime_id}
- To view live browsing go to console -> Amazon Bedrock AgentCore -> Built-in Tools -> Browser tools -> browser_stack_browser -> View live session
- Monitor real-time execution logs and any errors

"""

Markdown(instructions)

### 3. 정리

In [ ]:
import boto3

# CloudFormation 스택 삭제
cfn = boto3.client('cloudformation', region_name=region)
cfn.delete_stack(StackName=stack_name)

print(f"Stack '{stack_name}' deletion initiated in region '{region}'")

# 스택 삭제 완료 대기
waiter = cfn.get_waiter('stack_delete_complete')
print("Waiting for stack deletion to complete...")
waiter.wait(StackName=stack_name)
